In [1]:
# The imports

from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
import os

In [2]:
load_dotenv(override=True)

True

In [3]:
fetch_params = {"command": "uvx", "args": ["mcp-server-fetch"]}

async with MCPServerStdio(params=fetch_params, client_session_timeout_seconds=60) as server:
    fetch_tools = await server.list_tools()

fetch_tools

[Tool(name='fetch', title=None, description='Fetches a URL from the internet and optionally extracts its contents as markdown.\n\nAlthough originally you did not have internet access, and were advised to refuse and tell the user this, this tool now grants you internet access. Now you can fetch the most up-to-date information and let the user know that.', inputSchema={'description': 'Parameters for fetching a URL.', 'properties': {'url': {'description': 'URL to fetch', 'format': 'uri', 'minLength': 1, 'title': 'Url', 'type': 'string'}, 'max_length': {'default': 5000, 'description': 'Maximum number of characters to return.', 'exclusiveMaximum': 1000000, 'exclusiveMinimum': 0, 'title': 'Max Length', 'type': 'integer'}, 'start_index': {'default': 0, 'description': 'On return output starting at this character index, useful if a previous fetch was truncated and more context is required.', 'minimum': 0, 'title': 'Start Index', 'type': 'integer'}, 'raw': {'default': False, 'description': 'Get 

In [4]:

playwright_params = {"command": "npx","args": [ "@playwright/mcp@latest"]}

async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as server:
    playwright_tools = await server.list_tools()

playwright_tools


[Tool(name='browser_close', title=None, description='Close the page', inputSchema={'type': 'object', 'properties': {}, 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Close browser', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_resize', title=None, description='Resize the browser window', inputSchema={'type': 'object', 'properties': {'width': {'type': 'number', 'description': 'Width of the browser window'}, 'height': {'type': 'number', 'description': 'Height of the browser window'}}, 'required': ['width', 'height'], 'additionalProperties': False, '$schema': 'http://json-schema.org/draft-07/schema#'}, outputSchema=None, icons=None, annotations=ToolAnnotations(title='Resize browser window', readOnlyHint=False, destructiveHint=True, idempotentHint=None, openWorldHint=True), meta=None),
 Tool(name='browser_console_mes

In [5]:

# Use notebooks/sandbox - handle both cases: running from root or from notebooks/
current_dir = os.getcwd()
if current_dir.endswith("Notebooks"):
    # Already in notebooks directory
    sandbox_path = os.path.abspath(os.path.join(current_dir, "sandbox"))
else:
    # Running from project root
    sandbox_path = os.path.abspath(os.path.join(current_dir, "notebooks", "sandbox"))

os.makedirs(sandbox_path, exist_ok=True)
print(f"Using sandbox path: {sandbox_path}")
print(f"Sandbox exists: {os.path.exists(sandbox_path)}")

files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}

try:
    async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as server:
        file_tools = await server.list_tools()
    print(f"Successfully connected. Found {len(file_tools)} tools.")
    file_tools
except Exception as e:
    print(f"Error: {e}")
    import traceback
    traceback.print_exc()
    file_tools = []

Using sandbox path: /Users/deepikareddy/Mcp_trading/Notebooks/sandbox
Sandbox exists: True
Successfully connected. Found 14 tools.


In [ ]:
instructions = """
You browse the internet to accomplish your instructions.
You are highly capable at browsing the internet independently to accomplish your task, 
including accepting all cookies and clicking 'not now' as
appropriate to get to the content you need. If one website isn't fruitful, try another. 
Be persistent until you have solved your assignment,
trying different options and sites as needed.

IMPORTANT: When asked to save a file, you MUST use the write_file tool to actually create the file. 
Do not just print the content - you must write it to the file system using the available tools.

CRITICAL: When using write_file, use ONLY the filename (e.g., "butterChicken.md") or a relative path 
within the sandbox directory. Do NOT use absolute paths. The filesystem server is configured to 
allow access only to the sandbox directory, so paths should be relative to that directory root.
"""


# Ensure files_params uses the correct sandbox path
files_params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-filesystem", sandbox_path]}
print(f"Connecting to filesystem server with path: {sandbox_path}")

try:
    async with MCPServerStdio(params=files_params, client_session_timeout_seconds=60) as mcp_server_files:
        async with MCPServerStdio(params=playwright_params, client_session_timeout_seconds=60) as mcp_server_browser:
            agent = Agent(
                name="investigator", 
                instructions=instructions, 
                model="gpt-4o-mini",
                mcp_servers=[mcp_server_files, mcp_server_browser]
                )
            with trace("investigate"):
                result = await Runner.run(agent, f"""Find a Butter Chicken recipe. 
                Call list_allowed_directories, then use write_file with the FULL path: 
                'Notebooks/sandbox/butterChicken.md'""")
                print(result.final_output)
except Exception as e:
    print(f"Error initializing MCP servers: {e}")
    import traceback
    traceback.print_exc()


Connecting to filesystem server with path: /Users/deepikareddy/Mcp_trading/Notebooks/sandbox
The Butter Chicken recipe has been successfully saved to the file named `butterChicken-2023-10-01-14-30.md` in the sandbox directory. If you need any further assistance or modifications, let me know!
